In [ ]:
# 2_vae_model.ipynb

import tensorflow as tf
from tensorflow.keras import layers

# Define the encoder
latent_dim = 16

encoder_input = layers.Input(shape=(input_dim,))
x = layers.Dense(64, activation='relu')(encoder_input)
x = layers.Dense(32, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)

# Sampling layer
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(32, activation='relu')(decoder_input)
x = layers.Dense(64, activation='relu')(x)
decoder_output = layers.Dense(input_dim, activation='sigmoid')(x)

# Define VAE model
encoder = tf.keras.Model(encoder_input, [z_mean, z_log_var, z], name='encoder')
decoder = tf.keras.Model(decoder_input, decoder_output, name='decoder')

outputs = decoder(encoder(encoder_input)[2])
vae = tf.keras.Model(encoder_input, outputs, name='vae')

# Compile the VAE model
vae.compile(optimizer='adam', loss='mse')
vae.summary()
